In [1]:
import sys
import random
import torch
import importlib
from tensorboardX import SummaryWriter
import torch.nn.utils.rnn as rnn_utils
import pickle
import tqdm
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque
sys.path.append(".")
from dataUtilsV0 import *
import json
from RDM_Model import *
from CM_Model import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

load glove finished


In [2]:

class LayerNormLSTMCell(nn.LSTMCell):
    def __init__(self, input_size, hidden_size, dropout=0.0, bias=True, use_layer_norm=True):
        super().__init__(input_size, hidden_size, bias)
        self.use_layer_norm = use_layer_norm
        if self.use_layer_norm:
            self.ln_ih = nn.LayerNorm(4 * hidden_size)
            self.ln_hh = nn.LayerNorm(4 * hidden_size)
            self.ln_ho = nn.LayerNorm(hidden_size)
        # DropConnect on the recurrent hidden to hidden weight
        self.dropout = dropout

    def forward(self, input, hidden=None):
        self.check_forward_input(input)
        if hidden is None:
            hx = input.new_zeros(input.size(0), self.hidden_size, requires_grad=False)
            cx = input.new_zeros(input.size(0), self.hidden_size, requires_grad=False)
        else:
            hx, cx = hidden
        self.check_forward_hidden(input, hx, '[0]')
        self.check_forward_hidden(input, cx, '[1]')

        weight_hh = nn.functional.dropout(self.weight_hh, p=self.dropout, training=self.training)
        if self.use_layer_norm:
            gates = self.ln_ih(F.linear(input, self.weight_ih, self.bias_ih)) \
                     + self.ln_hh(F.linear(hx, weight_hh, self.bias_hh))
        else:
            gates = F.linear(input, self.weight_ih, self.bias_ih) \
                    + F.linear(hx, weight_hh, self.bias_hh)

        i, f, c, o = gates.chunk(4, 1)
        i_ = torch.sigmoid(i)
        f_ = torch.sigmoid(f)
        c_ = torch.tanh(c)
        o_ = torch.sigmoid(o)
        cy = (f_ * cx) + (i_ * c_)
        if self.use_layer_norm:
            hy = o_ * self.ln_ho(torch.tanh(cy))
        else:
            hy = o_ * torch.tanh(cy)
        return hy, cy

class LayerNormLSTM(nn.Module):
    def __init__(self,
                 input_size,
                 hidden_size,
                 num_layers=1,
                 dropout=0.0,
                 weight_dropout=0.0,
                 bias=True,
                 bidirectional=False,
                 use_layer_norm=True):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # using variational dropout
        self.dropout = dropout
        self.bidirectional = bidirectional

        num_directions = 2 if bidirectional else 1
        self.hidden0 = nn.ModuleList([
            LayerNormLSTMCell(input_size=(input_size if layer == 0 else hidden_size * num_directions),
                              hidden_size=hidden_size, dropout=weight_dropout, bias=bias, use_layer_norm=use_layer_norm)
            for layer in range(num_layers)
        ])

        if self.bidirectional:
            self.hidden1 = nn.ModuleList([
                LayerNormLSTMCell(input_size=(input_size if layer == 0 else hidden_size * num_directions),
                                  hidden_size=hidden_size, dropout=weight_dropout, bias=bias, use_layer_norm=use_layer_norm)
                for layer in range(num_layers)
            ])

    def copy_parameters(self, rnn_old):
        for param in rnn_old.named_parameters():
            name_ = param[0].split("_")
            layer = int(name_[2].replace("l", ""))
            sub_name = "_".join(name_[:2])
            if len(name_) > 3:
                self.hidden1[layer].register_parameter(sub_name, param[1])
            else:
                self.hidden0[layer].register_parameter(sub_name, param[1])

    def forward(self, input, hidden=None, seq_lens=None):
        seq_len, batch_size, _ = input.size()
        num_directions = 2 if self.bidirectional else 1
        if hidden is None:
            hx = input.new_zeros(self.num_layers * num_directions, batch_size, self.hidden_size, requires_grad=False)
            cx = input.new_zeros(self.num_layers * num_directions, batch_size, self.hidden_size, requires_grad=False)
        else:
            hx, cx = hidden

        ht = []
        for i in range(seq_len):
            ht.append([None] * (self.num_layers * num_directions))
        ct = []
        for i in range(seq_len):
            ct.append([None] * (self.num_layers * num_directions))

        seq_len_mask = input.new_ones(batch_size, seq_len, self.hidden_size, requires_grad=False)
        if seq_lens != None:
            for i, l in enumerate(seq_lens):
                seq_len_mask[i, l:, :] = 0
        seq_len_mask = seq_len_mask.transpose(0, 1)

        if self.bidirectional:
            # if use cuda, change 'torch.LongTensor' to 'torch.cuda.LongTensor'
            indices_ = (torch.LongTensor(seq_lens) - 1).unsqueeze(1).unsqueeze(0).unsqueeze(0).repeat(
                [1, 1, 1, self.hidden_size])
            # if use cuda, change 'torch.LongTensor' to 'torch.cuda.LongTensor'
            indices_reverse = torch.LongTensor([0] * batch_size).unsqueeze(1).unsqueeze(0).unsqueeze(0).repeat(
                [1, 1, 1, self.hidden_size])
            indices = torch.cat((indices_, indices_reverse), dim=1)
            hy = []
            cy = []
            xs = input
            # Variational Dropout
            if not self.training or self.dropout == 0:
                dropout_mask = input.new_ones(self.num_layers, 2, batch_size, self.hidden_size)
            else:
                dropout_mask = input.new(self.num_layers, 2, batch_size, self.hidden_size).bernoulli_(1 - self.dropout)
                dropout_mask = Variable(dropout_mask, requires_grad=False) / (1 - self.dropout)

            for l, (layer0, layer1) in enumerate(zip(self.hidden0, self.hidden1)):
                l0, l1 = 2 * l, 2 * l + 1
                h0, c0, h1, c1 = hx[l0], cx[l0], hx[l1], cx[l1]
                for t, (x0, x1) in enumerate(zip(xs, reversed(xs))):
                    ht_, ct_ = layer0(x0, (h0, c0))
                    ht[t][l0] = ht_ * seq_len_mask[t]
                    ct[t][l0] = ct_ * seq_len_mask[t]
                    h0, c0 = ht[t][l0], ct[t][l0]
                    t = seq_len - 1 - t
                    ht_, ct_ = layer1(x1, (h1, c1))
                    ht[t][l1] = ht_ * seq_len_mask[t]
                    ct[t][l1] = ct_ * seq_len_mask[t]
                    h1, c1 = ht[t][l1], ct[t][l1]

                xs = [torch.cat((h[l0]*dropout_mask[l][0], h[l1]*dropout_mask[l][1]), dim=1) for h in ht]
                ht_temp = torch.stack([torch.stack([h[l0], h[l1]]) for h in ht])
                ct_temp = torch.stack([torch.stack([c[l0], c[l1]]) for c in ct])
                if len(hy) == 0:
                    hy = torch.stack(list(ht_temp.gather(dim=0, index=indices).squeeze(0)))
                else:
                    hy = torch.cat((hy, torch.stack(list(ht_temp.gather(dim=0, index=indices).squeeze(0)))), dim=0)
                if len(cy) == 0:
                    cy = torch.stack(list(ct_temp.gather(dim=0, index=indices).squeeze(0)))
                else:
                    cy = torch.cat((cy, torch.stack(list(ct_temp.gather(dim=0, index=indices).squeeze(0)))), dim=0)
            y  = torch.stack(xs)
        else:
            # if use cuda, change 'torch.LongTensor' to 'torch.cuda.LongTensor'
            indices = (torch.cuda.LongTensor(seq_lens) - 1).unsqueeze(1).unsqueeze(0).unsqueeze(0).repeat(
                [1, self.num_layers, 1, self.hidden_size])
            h, c = hx, cx
            # Variational Dropout
            if not self.training or self.dropout == 0:
                dropout_mask = input.new_ones(self.num_layers, batch_size, self.hidden_size)
            else:
                dropout_mask = input.new(self.num_layers, batch_size, self.hidden_size).bernoulli_(1 - self.dropout)
                dropout_mask = Variable(dropout_mask, requires_grad=False) / (1 - self.dropout)

            for t, x in enumerate(input):
                for l, layer in enumerate(self.hidden0):
                    ht_, ct_ = layer(x, (h[l], c[l]))
                    ht[t][l] = ht_ * seq_len_mask[t]
                    ct[t][l] = ct_ * seq_len_mask[t]
                    x = ht[t][l] * dropout_mask[l]
                ht[t] = torch.stack(ht[t])
                ct[t] = torch.stack(ct[t])
                h, c = ht[t], ct[t]
            y = torch.stack([h[-1]*dropout_mask[-1] for h in ht])
            hy = torch.stack(list(torch.stack(ht).gather(dim=0, index=indices).squeeze(0)))
            cy = torch.stack(list(torch.stack(ct).gather(dim=0, index=indices).squeeze(0)))

        return y, (hy, cy)


# ### 模型训练与测试
class pooling_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(pooling_layer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def forward(self, inputs, cuda=True):
        inputs_sent = [torch.cat([self.linear(sent_tensor.cuda() if cuda else sent_tensor).max(axis=0)[0].unsqueeze(0) for sent_tensor in seq]) for seq in inputs]
        seqs = torch.nn.utils.rnn.pad_sequence(inputs_sent, batch_first=True)
        return seqs

class RDM_Model(nn.Module):
    def __init__(self, word_embedding_dim, sent_embedding_dim, hidden_dim, dropout_prob):
        super(RDM_Model, self).__init__()
        self.embedding_dim = sent_embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_model = nn.GRU(word_embedding_dim, 
                                self.hidden_dim, 
                                batch_first=True, 
                                dropout=dropout_prob
                            )
        self.DropLayer = nn.Dropout(dropout_prob)

    def forward(self, input_x): 
        """
        input_x: [batchsize, max_seq_len, sentence_embedding_dim] 
        x_len: [batchsize]
        init_states: [batchsize, hidden_dim]
        """
        batchsize, max_seq_len, emb_dim = input_x.shape
        init_states = torch.zeros([1, batchsize, self.hidden_dim], dtype=torch.float32).cuda()
        try:
            df_outputs, df_last_state = self.gru_model(input_x, init_states)
        except:
            print("Error:", pool_feature.shape, init_states.shape)
            raise
        return df_outputs

class RDM_Model_V1(nn.Module):
    def __init__(self, word_embedding_dim, sent_embedding_dim, hidden_dim, dropout_prob):
        super(RDM_Model_V1, self).__init__()
        self.embedding_dim = sent_embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_model = LayerNormLSTM(word_embedding_dim, 
                                self.hidden_dim, 
                                dropout=dropout_prob
                            )

    def forward(self, input_x, seq_lens): 
        """
        input_x: [batchsize, max_seq_len, sentence_embedding_dim] 
        x_len: [batchsize]
        init_states: [batchsize, hidden_dim]
        """
        batchsize, max_seq_len, emb_dim = input_x.shape
        h0 = torch.zeros([1, batchsize, self.hidden_dim], dtype=torch.float32).cuda()
        c0 = torch.zeros([1, batchsize, self.hidden_dim], dtype=torch.float32).cuda()
        df_outputs, (df_last_state, df_last_cell) = self.gru_model(input_x.transpose(0, 1), (h0, c0), seq_lens)
        return df_outputs.transpose(0, 1), df_last_state.transpose(0, 1), df_last_cell.transpose(0, 1)

class CM_Model_V1(nn.Module):
    def __init__(self, hidden_dim, action_num):
        super(CM_Model_V1, self).__init__()
        self.hidden_dim = hidden_dim
        self.action_num = action_num
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_state):
        """
        rdm_state: [batchsize, hidden_dim]
        """
        batchsize, hidden_dim = rdm_state.shape
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
                rdm_state
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop

class CM_Model(nn.Module):
    def __init__(self, sentence_embedding_dim, hidden_dim, action_num):
        super(CM_Model, self).__init__()
        self.sentence_embedding_dim = sentence_embedding_dim
        self.hidden_dim = hidden_dim
        self.action_num = action_num
#         self.PoolLayer = pooling_layer(self.embedding_dim, 
#                                             self.hidden_dim)
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_model, rl_input, rl_state):
        """
        rl_input: [batchsize, max_word_num, sentence_embedding_dim]
        rl_state: [1, batchsize, hidden_dim]
        """
        assert(rl_input.ndim==3)
        batchsize, max_word_num, embedding_dim = rl_input.shape
        rl_output, rl_new_state = rdm_model.gru_model(
                                            rl_input, 
                                            rl_state
                                        )
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
#                 rl_state.reshape([len(rl_input), self.hidden_dim]) #it is not sure to take rl_state , rather than rl_output, as the feature
                rl_output.reshape(
                    [len(rl_input), self.hidden_dim]
                )
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop, rl_new_state


# In[13]:


load_data_fast()

rdm_model = RDM_Model(300, 300, 256, 0.2).cuda()
sent_pooler = pooling_layer(300, 300).cuda()
rdm_classifier = nn.Linear(256, 2).cuda()
cm_model = CM_Model_V1(256, 2).cuda()

log_dir = os.path.join(sys.path[0], "ERD/")

with open("../../config.json", "r") as cr:
    dic = json.load(cr)

class adict(dict):
    ''' Attribute dictionary - a convenience data structure, similar to SimpleNamespace in python 3.3
        One can use attributes to read/write dictionary content.
    '''
    def __init__(self, *av, **kav):
        dict.__init__(self, *av, **kav)
        self.__dict__ = self

FLAGS = adict(dic)

max_sent: 64 ,  max_seq_len: 346
5802 data loaded


/home/hadoop/.conda/envs/torch_B/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [3]:
pretrained_file = "ERD/ERD_best.pkl"
if os.path.exists(pretrained_file):
    checkpoint = torch.load(pretrained_file)
    sent_pooler.load_state_dict(checkpoint['sent_pooler'])
    rdm_model.load_state_dict(checkpoint["rmdModel"])
    rdm_classifier.load_state_dict(checkpoint["rdm_classifier"])
else:
    TrainRDMModel(rdm_model, sent_pooler, rdm_classifier, 
                    t_steps=5000, stage=0, new_data_len=[], valid_new_len=[], logger=None, 
                        log_dir=log_dir, cuda=True)


In [4]:
del valid_data_ID
del valid_data_len
del valid_data_y

In [5]:
from dataUtilsV0 import valid_data_ID
from dataUtilsV0 import valid_data_len
from dataUtilsV0 import valid_data_y

In [6]:
stage = 0
t_rw = 0.5 
t_steps = 10000  
logger = None
cuda=True

In [10]:
batch_size = 40
t_acc = 0.9
gamma = 1.0
lambda1 = -1.0
lambda2 = 0.0 #regularizer
sum_loss = 0.0
sum_acc = 0.0
t_acc = 0.9
ret_acc = 0.0
init_states = torch.zeros([1, batch_size, rdm_model.hidden_dim], dtype=torch.float32).cuda()
weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
loss_fn = nn.CrossEntropyLoss(weight=weight, reduction='mean')
optim = torch.optim.Adam([
                            {'params': sent_pooler.parameters(), 'lr': 2e-5},
                            {'params': rdm_model.parameters(), 'lr': 2e-5},
                            {'params': rdm_classifier.parameters(), 'lr': 2e-5},
                            {'params': cm_model.parameters(), 'lr':2e-5}
                         ]
)

writer = SummaryWriter(log_dir, filename_suffix="_ERD_CM_stage_%3d"%stage)
best_valid_acc = 0.0
rw_arr = np.zeros(10)
len_arr = np.zeros(10)

In [11]:
weight1 = torch.tensor([1.0, 2.0], dtype=torch.float32).cuda()
loss_fn_1 = nn.CrossEntropyLoss(weight=weight, reduction="none")

In [12]:
for step in range(100):
    x, x_len, y = get_df_batch(step*batch_size, batch_size)        
    seq = sent_pooler(x)
    rdm_hiddens = rdm_model(seq)
    batchsize, max_seq_len, hidden_dim = rdm_hiddens.shape
    rdm_outs = torch.stack(
        [ rdm_hiddens[i][x_len[i]-1] for i in range(batchsize)] 
    )
    stopScore, isStop = cm_model(
        rdm_outs
    )
#     rdm_preds = rdm_scores.argmax(axis=1)
    y_label = torch.tensor(y).argmax(axis=1).cuda() if cuda else torch.tensor(y).argmax(axis=1)
    acc = accuracy_score(y_label.cpu().numpy(), isStop.cpu().numpy())
    loss = loss_fn(stopScore, y_label)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if step%10 == 0:  
        print('*****Optimizing PG***** %3d | %d , train_loss/Expected Reward = %6.8f/%6.7f,  RDM_Loss/RDM Accuracy = %6.8f/%6.7f, mean_len = %2.3f'% (step, t_steps, 
                0.0, 0 , loss, acc, 0.0))

*****Optimizing PG*****   0 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.66222590/0.6750000, mean_len = 0.000
*****Optimizing PG*****  10 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.62323058/0.7750000, mean_len = 0.000
*****Optimizing PG*****  20 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.57268417/0.9750000, mean_len = 0.000
*****Optimizing PG*****  30 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.55740982/0.9250000, mean_len = 0.000
*****Optimizing PG*****  40 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.49361047/0.9750000, mean_len = 0.000
*****Optimizing PG*****  50 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.48326612/0.9500000, mean_len = 0.000
*****Optimizing PG*****  60 | 10000 , train_loss/Expected Reward = 0.00000000/0.00

In [13]:
for step in range(500):
    x, x_len, y = get_df_batch(step*batch_size, batch_size)        
    seq = sent_pooler(x)
    rdm_hiddens = rdm_model(seq)
    batchsize, max_seq_len, hidden_dim = rdm_hiddens.shape
    rdm_outs = torch.stack(
        [ rdm_hiddens[i][x_len[i]-1] for i in range(batchsize)] 
    )
    rdm_scores = rdm_classifier(
        rdm_outs
    )
    rdm_preds = rdm_scores.argmax(axis=1)
    y_label = torch.tensor(y).argmax(axis=1).cuda() if cuda else torch.tensor(y).argmax(axis=1)
    acc = accuracy_score(y_label.cpu().numpy(), rdm_preds.cpu().numpy())
    loss = loss_fn(rdm_scores, y_label)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if step%10 == 0:  
        print('*****Optimizing RDM***** %3d | %d , train_loss/Expected Reward = %6.8f/%6.7f,  RDM_Loss/RDM Accuracy = %6.8f/%6.7f, mean_len = %2.3f'% (step, t_steps, 
                0.0, 0 , loss, acc, 0.0))

*****Optimizing RDM*****   0 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.07372034/0.9750000, mean_len = 0.000
*****Optimizing RDM*****  10 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.00804892/1.0000000, mean_len = 0.000
*****Optimizing RDM*****  20 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.01420731/1.0000000, mean_len = 0.000
*****Optimizing RDM*****  30 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.14986932/0.9750000, mean_len = 0.000
*****Optimizing RDM*****  40 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.01985872/1.0000000, mean_len = 0.000
*****Optimizing RDM*****  50 | 10000 , train_loss/Expected Reward = 0.00000000/0.0000000,  RDM_Loss/RDM Accuracy = 0.11529309/0.9500000, mean_len = 0.000
*****Optimizing RDM*****  60 | 10000 , train_loss/Expected Reward = 0.000000

In [14]:
mean_rw = 0.0
update_lambda = 0.2
gamma = 0.9
max_rw_len = 2000

In [15]:
losses = np.zeros(10)

In [16]:
batch_size = 40
t_acc = 0.9
gamma = 1.0
lambda1 = -1.0
lambda2 = 0.0 #regularizer
sum_loss = 0.0
sum_acc = 0.0
t_acc = 0.9
ret_acc = 0.0
init_states = torch.zeros([1, batch_size, rdm_model.hidden_dim], dtype=torch.float32).cuda()
weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
loss_fn = nn.CrossEntropyLoss(weight=weight, reduction='mean')
optim = torch.optim.Adam([
                            {'params': sent_pooler.parameters(), 'lr': 2e-7},
                            {'params': rdm_model.parameters(), 'lr': 2e-7},
                            {'params': rdm_classifier.parameters(), 'lr': 2e-7},
                            {'params': cm_model.parameters(), 'lr':2e-7}
                         ]
)

writer = SummaryWriter(log_dir, filename_suffix="_ERD_CM_stage_%3d"%stage)
best_valid_acc = 0.0
rw_arr = np.zeros(10)
len_arr = np.zeros(10)

In [17]:
for step in range(5000):
    x, x_len, y = get_df_batch(step*batch_size, batch_size)        
    
    seq = sent_pooler(x)
    rdm_hiddens = rdm_model(seq)
    batchsize, max_seq_len, hidden_dim = rdm_hiddens.shape
    stopScore, isStop = cm_model(rdm_hiddens.reshape(-1, hidden_dim))
    isStop = isStop.reshape([batchsize, max_seq_len])
    stopProb = stopScore.reshape([batchsize, max_seq_len, -1]).softmax(axis=-1)         
    
    stop_idxs = isStop.detach()
    x_mask = torch.zeros(batchsize, max_seq_len)
    prob_mask = torch.ones([batchsize, max_seq_len, 2]).cuda()
    for i, l in enumerate(x_len):
        x_mask[i, :l].fill_(1.0)
        stop_idxs[i, l-1] = 1.0
    sampling_mask = torch.zeros(batchsize, max_seq_len).cuda()
    
    len_list = [x_len[i] - torch.arange(x_len[i]) for i in range(batchsize)]
    pad_length = torch.nn.utils.rnn.pad_sequence(len_list, batch_first=True, padding_value=1.0).cuda()
    
    sum_len = 0.0
    sampling_mask = torch.zeros(batchsize, max_seq_len).cuda()
    
    stop_error_pulish = torch.zeros(batchsize, max_seq_len).cuda()
    for j in range(len(x_len)):
        start = random.randint(0, x_len[j]-1)
        end = x_len[j]
        for t in range(start, x_len[j]):
            rnd = random.random()
            if stopProb[j][t][0] < stopProb[j][t][1]:
                prob_mask[j][t][0] = 0.0
                rnd2 = random.random()
                if y[j][0] == 1:
                    stop_error_pulish[j][t] = -100.0
                else:
                    stop_error_pulish[j][t] = 10.0
                if  rnd2 > stopProb[j][t][1]:
                    end = t
                    break
            else:
                prob_mask[j][t][1] = 0.0
        sampling_mask[j, start:end].fill_(1.0)
        sum_len += (end - start+1)*1.0/x_len[j]

    stop_vals = (stopProb*prob_mask).sum(axis=-1)
    log_stop_vals = torch.log(stop_vals)
    delay_punish = -1.0*torch.arange(1, max_seq_len+1, 1).float().cuda()
    preds = rdm_classifier(rdm_hiddens.reshape(-1, hidden_dim)).reshape(batchsize, max_seq_len, -1)
    yy = torch.tensor(y).unsqueeze(1).repeat((1, max_seq_len, 1)).reshape([-1, 2]).cuda()
    stop_loss = loss_fn_1(preds.reshape(-1, 2), yy.argmax(axis=1)).reshape([batchsize, max_seq_len])*x_mask.cuda()
    accum_loss = torch.stack([stop_loss[:, i:].sum(axis=1) for i in range(max_seq_len)]).transpose(0, 1)
    mean_loss = accum_loss/pad_length
    
    loss_pulish = stop_idxs.float() * mean_loss
    masked_delay_pulish = delay_punish*(1.0-stop_idxs).float()
    
    stopreward = (loss_pulish - masked_delay_pulish - stop_error_pulish)*sampling_mask
    
    log_sampling_probs = -1.0*log_stop_vals*sampling_mask
#     mean_reward = log_sampling_probs *stopreward
#     esti_reward = mean_reward.sum(axis=1).mean()  
    
    if stopreward.sum(axis=1).mean() < losses.mean():
        print("********optimize policy******")
        mean_reward = log_sampling_probs *stopreward.detach()
        esti_reward = mean_reward.sum(axis=1).mean()
    else:
        print("--------optimize reward-----")
        mean_reward = stopreward * log_sampling_probs.detach()
        esti_reward = mean_reward.sum(axis=1).mean()

    losses[step%10] = float(stopreward.sum(axis=1).mean())
    rw_arr[step%10] = float(esti_reward)
    len_arr[step%10] = float(sum_len/10.0)
    optim.zero_grad()
    esti_reward.backward()
    optim.step()
    if step%10 == 0:  
        print('%3d | %d , reward/Expected Reward = %6.8f/%6.7f,  RDM_Loss/RDM Accuracy = %6.8f/%6.7f, mean_len = %2.3f'             % (step, t_steps, 
                losses.mean(), rw_arr.mean(), loss, acc, len_arr.mean()
                ))
        print("isStop:", [isStop[idx][:l].cpu().tolist() for idx, l in enumerate(x_len)])

--------optimize reward-----
  0 | 10000 , reward/Expected Reward = 1.80838718/0.7297578,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 0.149
isStop: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
--------optimize reward-----
********optimize policy******
********optimize policy******
--------optimize reward-----
--------optimize reward-----
 30 | 10000 , reward/Expected Reward = 88.19235687/32.4459086,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.369
isStop: [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
 60 | 10000 , reward/Expected Reward = 71.42082891/25.9917644,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.347
isStop: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
 90 | 10000 , reward/Expected Reward = 88.21032500/32.7533732,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.357
isStop: [[0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
********optimize policy******
120 | 10000 , reward/Expected Reward = 50.91423507/19.7206781,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.463
isStop: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 

...

--------optimize reward-----
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
********optimize policy******
--------optimize reward-----
4870 | 10000 , reward/Expected Reward = 132.19894371/41.9929796,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.638
isStop: [[0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

********optimize policy******
--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
4900 | 10000 , reward/Expected Reward = 161.56783447/54.1534076,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.730
isStop: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
--------optimize reward-----
--------optimize reward-----
********optimize policy******
4930 | 10000 , reward/Expected Reward = 114.06423569/33.0346560,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.666
isStop: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

********optimize policy******
--------optimize reward-----
4950 | 10000 , reward/Expected Reward = 148.27902317/44.7326468,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.739
isStop: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0

--------optimize reward-----
********optimize policy******
--------optimize reward-----
--------optimize reward-----
********optimize policy******
********optimize policy******
********optimize policy******
********optimize policy******
--------optimize reward-----
********optimize policy******
4980 | 10000 , reward/Expected Reward = 151.05930672/46.7701837,  RDM_Loss/RDM Accuracy = 0.01528603/1.0000000, mean_len = 1.669
isStop: [[0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [18]:
x, x_len, y = get_df_batch(0*batch_size, batch_size)        
seq = sent_pooler(x)
rdm_hiddens = rdm_model(seq)
batchsize, max_seq_len, hidden_dim = rdm_hiddens.shape
stopScore, isStop = cm_model(rdm_hiddens.reshape(-1, hidden_dim))
isStop = isStop.reshape([batchsize, max_seq_len])
for idx, l in enumerate(x_len):
    if y[idx][1] == 1:
        print("#####isStop %3d:"%idx, isStop[idx][:l].cpu().tolist())
        print("\n")
    else:
        print("****isStop %3d:"%idx, isStop[idx][:l].cpu().tolist())
        print("\n")

#####isStop   0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#####isStop   1: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#####isStop   2: [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


****isStop   3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#####isStop   4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


****isStop   5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#####isStop   6: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


#####isStop   7: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#####isStop   8: [1, 1, 1, 1, 1, 1]


#####isStop   9: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#####isStop  10: [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=int32)

In [36]:
mean_loss[0][:x_len[0]]

tensor([0.0755, 0.0417, 0.0338, 0.0323, 0.0308, 0.0294, 0.0262, 0.0261, 0.0260,
        0.0260, 0.0262, 0.0261, 0.0255, 0.0255, 0.0252, 0.0248, 0.0248, 0.0254,
        0.0249, 0.0251, 0.0253, 0.0238, 0.0239, 0.0236, 0.0233, 0.0234, 0.0241,
        0.0250, 0.0238, 0.0234, 0.0233, 0.0238, 0.0243, 0.0205, 0.0211, 0.0215,
        0.0221, 0.0221, 0.0215, 0.0193, 0.0176, 0.0173, 0.0117],
       device='cuda:0', grad_fn=<SliceBackward>)

In [27]:
x_len[0]

43

In [35]:
stop_loss[0][:x_len[0]]

tensor([1.4949, 0.3681, 0.0913, 0.0933, 0.0820, 0.1469, 0.0321, 0.0280, 0.0274,
        0.0170, 0.0312, 0.0434, 0.0255, 0.0359, 0.0368, 0.0233, 0.0096, 0.0378,
        0.0201, 0.0200, 0.0597, 0.0201, 0.0304, 0.0294, 0.0217, 0.0117, 0.0098,
        0.0428, 0.0296, 0.0246, 0.0175, 0.0182, 0.0626, 0.0148, 0.0182, 0.0167,
        0.0222, 0.0251, 0.0304, 0.0245, 0.0182, 0.0228, 0.0117],
       device='cuda:0', grad_fn=<SliceBackward>)

> __解决求梯度的思路__

> - 如果采样整个序列， 那么最后就会不合理
> - 如果是序列中间抽选呢，那就又涉及到在element上计算梯度的问题 

> 解决方法： 用矩阵来mask掉那些没有被抽样到的其它元素。

> 解决流程:

> 1. 计算stop_loss, 它会根据是否停止来进行mask， 如果没有停止，采样到的序列便也不会停止，那么stop_idxs上对应的元素是0，就能把分类误差给抹除掉
> 2. 计算delay_punish
> 3. 计算采样序列的损失，采样到的序列只是其中的某个部分，因此需要把这部分对应的元素值设为1， 其它元素为0， 那么最后矩阵相乘之后，得到的元素之和就是这个采样序列的损失

In [13]:
from dataUtilsV0 import data_len

In [15]:
erd_save_as = '%s/erdModel_test2.pkl'% (log_dir)
torch.save(
    {
        "sent_pooler":sent_pooler.state_dict(),
        "rmdModel":rdm_model.state_dict(),
        "rdm_classifier": rdm_classifier.state_dict(),
        "cm_model":cm_model.state_dict()
    },
    erd_save_as
)

> 以上对于模型的存储:
> - test2 表示存储的是交叉优化的模型

In [19]:
del data
del data_ID
del data_y
del data_len
del valid_data_ID
del valid_data_len
del valid_data_y

from dataUtilsV0 import data
from dataUtilsV0 import data_ID
from dataUtilsV0 import data_len
from dataUtilsV0 import data_y
from dataUtilsV0 import valid_data_ID
from dataUtilsV0 import valid_data_len
from dataUtilsV0 import valid_data_y

In [20]:
valid_new_len = get_new_len_on_valid_data(sent_pooler, rdm_model, cm_model, FLAGS, cuda=True)

In [21]:
s = (np.array(valid_data_len) - np.array(valid_new_len))/np.array(valid_data_len)

In [22]:
s.mean()

0.4086957612856111

In [23]:
accuracy_on_valid_data(rdm_model, sent_pooler, rdm_classifier)

(0.772, 0.9698275862068966, 0.677710843373494)

In [24]:
accuracy_on_valid_data(rdm_model, sent_pooler, rdm_classifier, new_data_len=valid_new_len)

(0.772, 0.961864406779661, 0.6837349397590361)

In [16]:
del accuracy_on_valid_data

In [25]:
load_data_fast()

max_sent: 64 ,  max_seq_len: 346
5802 data loaded


In [28]:
del data
del data_ID
del data_y
del data_len
del valid_data_ID
del valid_data_len
del valid_data_y

from dataUtilsV0 import data
from dataUtilsV0 import data_ID
from dataUtilsV0 import data_len
from dataUtilsV0 import data_y
from dataUtilsV0 import valid_data_ID
from dataUtilsV0 import valid_data_len
from dataUtilsV0 import valid_data_y

In [44]:
def accuracy_on_valid_data(rdm_model = None, sent_pooler = None, rdm_classifier=None, new_data_len=[], cuda=True):
    batch_size = 20
    t_steps = int(len(valid_data_ID)/batch_size)
    
    sum_acc = 0.0
    sum_pres = 0.0
    sum_recall = 0.0
    
    miss_vec = 0
    mts = 0
    hit_vec = 0
    if len(new_data_len) > 0:
        t_data_len = new_data_len
    else:
        t_data_len = valid_data_len
    labels = []
    preds = []
    for step in range(t_steps):
        data_x = []
        m_data_y = np.zeros([batch_size, 2], dtype=np.int32)
        m_data_len = np.zeros([batch_size], dtype=np.int32)
        for i in range(batch_size):
            m_data_y[i] = valid_data_y[mts]
            m_data_len[i] = t_data_len[mts]
            seq = []
            for j in range(t_data_len[mts]):
                sent = wordlist2wordvecs(data[valid_data_ID[mts]]['text'][j])
                sent_tensor = torch.tensor(np.stack(sent))
                seq.append(sent_tensor)
            data_x.append(seq)
            mts += 1
            if mts >= len(valid_data_ID): # read data looply
                mts = mts % len(valid_data_ID)
        
        
        if rdm_model is not None and sent_pooler is not None and rdm_classifier is not None:
            with torch.no_grad():
                seq = sent_pooler(data_x)
                rdm_hiddens = rdm_model(seq)
                batchsize, _, _ = rdm_hiddens.shape
                rdm_outs = torch.stack(
                    [ rdm_hiddens[i][m_data_len[i]-1] for i in range(batchsize)] 
#                     [ rdm_hiddens[i][0] for i in range(batchsize)] 
                )
                rdm_scores = rdm_classifier(
                    rdm_outs
                )
                rdm_preds = rdm_scores.argmax(axis=1)
                y_label = torch.tensor(m_data_y).argmax(axis=1).cuda() if cuda else torch.tensor(m_data_y).argmax(axis=1)
                preds.append(rdm_preds)
                labels.append(y_label)
    
    pred_array = torch.cat(preds).cpu().numpy()
    label_array = torch.cat(labels).cpu().numpy()
    return accuracy_score(y_true=label_array, y_pred=pred_array), precision_score(y_true=label_array, y_pred=pred_array), recall_score(y_true=label_array, y_pred=pred_array)

In [43]:
accuracy_on_valid_data(rdm_model, sent_pooler, rdm_classifier) # 使用第0条推特

(0.836, 0.8255208333333334, 0.9548192771084337)

In [45]:
accuracy_on_valid_data(rdm_model, sent_pooler, rdm_classifier) # 使用最后1条推特

(0.874, 0.8898550724637682, 0.9246987951807228)